## Import libraries

In [ ]:
!pip install pip install pyLDAvis

In [ ]:
!python -m pip install -U pyLDAvis


In [ ]:
!pip install scikit-learn==0.19.2

In [ ]:
!pip install language_tool_python

In [ ]:
!nvidia-smi

In [ ]:
!pip install -q -U watermark

In [ ]:
!pip install -qq transformers

In [ ]:
pip install transformers==3.4.0

In [ ]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
# import transformers as ppb
import re, nltk, spacy, string
import pandas as pd 
# from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pprint import pprint

import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

from plotly.offline import plot
import plotly.graph_objects as go
import plotly.express as px

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import warnings
warnings.filterwarnings('ignore')
# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

## Scrape Amazon Reviews

In [ ]:
# %%time
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd

# reviewlist = []

# def get_soup(url):
#     r = requests.get('http://localhost:8050/render.html', params={'url': url, 'wait': 2})
#     soup = BeautifulSoup(r.text, 'html.parser')
#     return soup


# def get_reviews(soup):
#     reviews = soup.find_all('div', {'data-hook': 'review'})
#     try:
#         for item in reviews:
#             review = {
#             'product': soup.title.text.replace('Amazon.co.uk:Customer reviews:', '').strip(),
#             'title': item.find('a', {'data-hook': 'review-title'}).text.strip(),
#             'rating':  float(item.find('i', {'data-hook': 'review-star-rating'}).text.replace('out of 5 stars', '').strip()),
#             'body': item.find('span', {'data-hook': 'review-body'}).text.strip(),
#             }
#             reviewlist.append(review)
#     except:
#         pass

# for x in range(1,500):
#     soup = get_soup(f'https://www.amazon.com/Under-Armour-Charged-Assert-Running/product-reviews/B08DXCSW56/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews{x}')
#     print(f'Getting page: {x}')
#     get_reviews(soup)
#     print(len(reviewlist))
#     if not soup.find('li', {'class': 'a-disabled a-last'}):
#         pass
#     else:
#         break


# print('Fin.')

In [ ]:

# df = pd.DataFrame(reviewlist)
# len(df)

In [ ]:
# df.to_excel('Shoes2.xlsx', index=False)

## Data Exploration

In [ ]:
!gdown --id 1S6qMioqPJjyBLpLVz4gmRTnJHnjitnuV
!gdown --id 1zdmewp7ayS4js4VtrJEHzAheSW-5NBZv

In [ ]:
df = pd.read_csv("reviews.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
sns.countplot(df.score)
plt.xlabel('review score');

In [ ]:
def to_sentiment(rating):
  rating = int(rating)
  if rating <= 2:
    return 0
  elif rating == 3:
    return 1
  else: 
    return 2

df['sentiment'] = df.score.apply(to_sentiment)

In [ ]:
class_names = ['negative', 'neutral', 'positive']

In [ ]:
ax = sns.countplot(df.sentiment)
plt.xlabel('review sentiment')
ax.set_xticklabels(class_names);

## Data Preprocessing
- Add special tokens to separate sentences and do classification
- Pass sequences of constant length (introduce padding)
- Create array of 0s (pad token) and 1s (real token) called *attention mask*

In [ ]:
from transformers import *


In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'

In [ ]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
sample_txt = 'When was I last outside? I am stuck at home for 2 weeks.'

In [ ]:
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f' Sentence: {sample_txt}')
print(f'   Tokens: {tokens}')
print(f'Token IDs: {token_ids}')

### Special Tokens

`[SEP]` - marker for ending of a sentence


In [ ]:
tokenizer.sep_token, tokenizer.sep_token_id

`[CLS]` - we must add this token to the start of each sentence, so BERT knows we're doing classification

In [ ]:
tokenizer.cls_token, tokenizer.cls_token_id

## special token for padding:

In [ ]:
tokenizer.pad_token, tokenizer.pad_token_id

## BERT understands tokens that were in the training set. Everything else can be 

---

encoded using the `[UNK]` (unknown) token:

In [ ]:
tokenizer.unk_token, tokenizer.unk_token_id

In [ ]:
encoding = tokenizer.encode_plus(
  sample_txt,
  max_length=32,
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',  # Return PyTorch tensors
)

encoding.keys()

In [ ]:
print(len(encoding['input_ids'][0]))
encoding['input_ids'][0]

In [ ]:
print(len(encoding['attention_mask'][0]))
encoding['attention_mask']

In [ ]:
tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])

## Choosing Sequence Length

BERT works with fixed-length sequences. We'll use a simple strategy to choose the max length. Let's store the token length of each review:



In [ ]:
token_lens = []

for txt in df.content:
  tokens = tokenizer.encode(txt, max_length=512)
  token_lens.append(len(tokens))

In [ ]:
sns.distplot(token_lens)
plt.xlim([0, 256]);
plt.xlabel('Token count');



```

```

Most of the reviews seem to contain less than 128 tokens, but we'll be on the safe side and choose a maximum length of 160.

In [ ]:
MAX_LEN = 160

In [ ]:
class GPReviewDataset(Dataset):

  def __init__(self, reviews, targets, tokenizer, max_len):
    self.reviews = reviews
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.reviews)
  
  def __getitem__(self, item):
    review = str(self.reviews[item])
    target = self.targets[item]

    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    return {
      'review_text': review,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [ ]:
## Split the data into training, validation and testing

df_train, df_test = train_test_split(df, test_size=0.1, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = GPReviewDataset(
    reviews=df.content.to_numpy(),
    targets=df.sentiment.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4
  )

In [ ]:
BATCH_SIZE = 32

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [ ]:
data = next(iter(train_data_loader))
data.keys()

In [ ]:
print(data['input_ids'].shape)
print(data['attention_mask'].shape)
print(data['targets'].shape)

## Sentiment Analysis Using BERT


In [ ]:
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
last_hidden_state, pooled_output = bert_model(
  input_ids=encoding['input_ids'], 
  attention_mask=encoding['attention_mask']
)

In [ ]:
last_hidden_state.shape

In [ ]:
bert_model.config.hidden_size

In [ ]:
class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [ ]:
model = SentimentClassifier(len(class_names))
model = model.to(device)

In [ ]:
input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)

print(input_ids.shape) # batch size x seq length
print(attention_mask.shape) # batch size x seq length

In [ ]:
from torch.nn import functional as F

F.softmax(model(input_ids, attention_mask), dim=1)

## Training

In [ ]:
EPOCHS = 20

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
def train_epoch(
  model, 
  data_loader, 
  loss_fn, 
  optimizer, 
  device, 
  scheduler, 
  n_examples
):
  model = model.train()

  losses = []
  correct_predictions = 0
  
  for d in data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)

    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )

    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)

    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
%%time

history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)

  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,    
    loss_fn, 
    optimizer, 
    device, 
    scheduler, 
    len(df_train)
  )

  print(f'Train loss {train_loss} accuracy {train_acc}')

  val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn, 
    device, 
    len(df_val)
  )

  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)

  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc

In [ ]:
plt.plot(history['train_acc'], label='train accuracy')
plt.plot(history['val_acc'], label='validation accuracy')

plt.title('Training history')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.ylim([0, 1]);

## Evaluation


In [ ]:
test_acc, _ = eval_model(
  model,
  test_data_loader,
  loss_fn,
  device,
  len(df_test)
)

test_acc.item()

In [ ]:
def get_predictions(model, data_loader):
  model = model.eval()
  
  review_texts = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in data_loader:

      texts = d["review_text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      probs = F.softmax(outputs, dim=1)

      review_texts.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(probs)
      real_values.extend(targets)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return review_texts, predictions, prediction_probs, real_values

In [ ]:
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(
  model,
  test_data_loader
)

In [ ]:
print(classification_report(y_test, y_pred, target_names=class_names))

In [ ]:
def show_confusion_matrix(confusion_matrix):
  hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
  hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
  hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
  plt.ylabel('True sentiment')
  plt.xlabel('Predicted sentiment');

cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
show_confusion_matrix(df_cm)

In [ ]:
idx = 2

review_text = y_review_texts[idx]
true_sentiment = y_test[idx]
pred_df = pd.DataFrame({
  'class_names': class_names,
  'values': y_pred_probs[idx]
})

In [ ]:
print("\n".join(wrap(review_text)))
print()
print(f'True sentiment: {class_names[true_sentiment]}')

In [ ]:
sns.barplot(x='values', y='class_names', data=pred_df, orient='h')
plt.ylabel('sentiment')
plt.xlabel('probability')
plt.xlim([0, 1]);

## Detect the fake reviews 


1.  Remove the reviews that have a 100% correct grammar 
2.  Remove the reviews that the title and the content is same
3.  Remove the reviews if the number of verbs > number of nouns in the review 



#stability selection


In [ ]:
df2 = pd.read_csv('/content/df.csv')  
df2

In [ ]:
df2 = df2.dropna()
df2 = df2.reset_index(drop=True)


In [ ]:
# from sklearn.linear_model import RandomizedLasso
# randomized_lasso = RandomizedLasso() 

In [ ]:
# pip install sentence_transformers

In [ ]:
# from sklearn.linear_model import RandomizedLasso
# randomized_lasso = RandomizedLasso() 
# import pandas as pd
# import numpy as np
# from matplotlib import pyplot as plt
# import seaborn as sns
# %matplotlib inline
# from sklearn.feature_selection import RFE, f_regression
# from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.ensemble import RandomForestRegressor

In [ ]:
# bert1 = list(df2['reviewText'])

In [ ]:
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('bert-base-nli-mean-tokens')
# embeddings = model.encode(review_text, show_progress_bar=True)

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# X = scaler.fit_transform(embeddings)

In [ ]:
# Y=  df2['sentiment'].array

In [ ]:
# from sklearn.linear_model import RandomizedLogisticRegression



# clf = RandomizedLogisticRegression(sample_fraction = 0.25,selection_threshold = 0.4, n_resampling = 200)
# clf.fit(X,Y)
# print (clf.get_support())

In [ ]:
# a = (clf.get_support(indices=True))
# print(len(a))

In [ ]:
# X = pd.DataFrame(X)

In [ ]:
# a = a.tolist()

In [ ]:
# New = pd.DataFrame()

In [ ]:

# iterator = 1
# for i in a:
#   if i in X.columns:
#     col_name = 'col' + str(i)
#     New.loc[:, col_name] = X[i] 
#     iterator += 1 


In [ ]:
# New

In [ ]:
# import language_tool_python
# tool = language_tool_python.LanguageTool('en-US')
# token = []
# neglected = []
# size = df2['body'].size
# for i in range (len(df2['body'])):
#     if  len(tool.check(df2['body'][i])) > 0  :
#         token.append(i)
#     else:
#         neglected.append(i)

# len(neglected)

In [ ]:
# for i in (neglected):
#   df2 =  df2.drop(i)
# df2 = df2.reset_index(drop=True)
# len(df2)

In [ ]:
# #1. different sentiment in review headline and review body

# remove_reviews = []
# # stores the list of review_id of fake reviews

# for i in range(len(df2['body'])):
#     #iterate through the whole dataset
    
#         if(( df2["title"][i] ) == ( df2["body"][i] ) ):
#           remove_reviews.append(i)
#             # checking if the sentiment of the body and the headline are not same
            
#             # remove_reviews.append(dataset["review_id"][i])
#             # append review_id to the list of fake reviews.

In [ ]:
# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

# remove_reviews = []
# for i in range(len(df2['body'])):
#     #iterate the whole dataset
    
#     words = nltk.word_tokenize(str(df2["body"][i]))
#     #storing the words from the reviews into the list
    
#     tagged_words = nltk.pos_tag(words)
#     # returns list of tuples of words along with their parts of speech
    
#     nouns_count = 0
#     verbs_count = 0
    
#     for j in range(len(tagged_words)):
#         #iterate through all the words

#         if(tagged_words[j][1].startswith("NN")):
#             nouns_count+=1
#             #counts the no. of nouns in the review

#         if(tagged_words[j][1].startswith("VB")):
#             verbs_count+=1
#             #counts the no. of verbs in the review

#     if(verbs_count>nouns_count):
#         #comparing the no. of verbs and nouns
#         remove_reviews.append(i)
#         #storing the review to be removed

In [ ]:
# print(len(remove_reviews))
# for i in(remove_reviews):
#     df2= df2.drop(i)

# df2 = df2.reset_index(drop=True)
# print(len(df2))

## Preprocessing the predictied data 

In [ ]:
#df2["review"] = df2["body"].str.lower()
df2["review"] = df2["reviewText"].str.lower()


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

In [ ]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df2["review"] = df2["review"].apply(lambda text: remove_stopwords(text))


In [ ]:
import string
import re
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df2["review"] = df2["review"].apply(lambda text: remove_punctuation(text))

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

df2["review"] = df2["review"].apply(lambda text: remove_emoji(text))

In [ ]:
def remove_urls(text):
  
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    try:
        return url_pattern.sub(r'', text)
    except:
        print(text)
    
df2["review"] = df2["review"].apply(lambda text: remove_urls(text))


In [ ]:
review_text = df2['reviewText'].tolist()

## Predicting product reviews into positive, negative and neutral 

In [ ]:
positive_reviews = []
negative_reviews = []
neutral_reviews = []

for i in range(len(review_text)):
  encoded_review = tokenizer.encode_plus(
      review_text[i],
      max_length=MAX_LEN,
      truncation=True,
      add_special_tokens=True,
      return_token_type_ids=False,
      padding='max_length',
      return_attention_mask=True,
      return_tensors='pt',)
  input_ids = encoded_review['input_ids'].to(device)
  attention_mask = encoded_review['attention_mask'].to(device)
  output = model(input_ids, attention_mask)
  _, prediction = torch.max(output, dim=1)
  if class_names[prediction] == "positive":
    positive_reviews.append(df2['review'][i])
  elif class_names[prediction] == "negative":
    negative_reviews.append(df2['review'][i])
  elif class_names[prediction] == "neutral":
    neutral_reviews.append(df2['review'][i])


In [ ]:
print(len(positive_reviews))
print(len(negative_reviews))
print(len(neutral_reviews))

In [ ]:
df_positive = pd.DataFrame(positive_reviews)
df_negative = pd.DataFrame(negative_reviews)
df_neutral = pd.DataFrame(neutral_reviews)


In [ ]:
df_positive1 = df_positive.to_csv("df_Positive.csv")
df_negative1 = df_negative.to_csv("df_Negative.csv")
df_neutral1 = df_neutral.to_csv("df_Neutral.csv")

In [ ]:
# positive_reviews1 = []
# negative_reviews2 = []
# neutral_reviews3 = []

# for i in range(0,50):
#   encoded_review = tokenizer.encode_plus(
#       review_text[i],
#       max_length=MAX_LEN,
#       add_special_tokens=True,
#       return_token_type_ids=False,
#       padding=True,
#       return_attention_mask=True,
#       return_tensors='pt',)
#   input_ids = encoded_review['input_ids'].to(device)
#   attention_mask = encoded_review['attention_mask'].to(device)
#   output = model(input_ids, attention_mask)
#   _, prediction = torch.max(output, dim=1)
#   if class_names[prediction] == "positive":
#     positive_reviews1.append(df2['review'][i])
#   elif class_names[prediction] == "negative":
#     negative_reviews2.append(df2['review'][i])
#   elif class_names[prediction] == "neutral":
#     neutral_reviews3.append(df2['review'][i])



## Topic modeling

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df2['review']))

print(data_words[:1])

In [ ]:
# Build the bigram and trigram models

bigram = gensim.models.Phrases(data_words) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=65)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

# data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])





In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:2])

In [ ]:
%%time

# Build LDA model
import numpy as np
from numpy.random import randint
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=42,
                                           update_every=25,
                                           chunksize=100,
                                           passes=30, 
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
print(type(lda_model.print_topics()))

In [ ]:

for i in range (4):
  words  = (lda_model.print_topics()[i])
  s = ','.join(str(v) for v in words)
  result = ''.join([i for i in s if not i.isdigit()])
  a= re.sub("[\"\']", "", result)
  a = a.replace('+', '')
  a = a.replace('.', '')
  a = a.replace('*', '')
  a = a.replace(',', '')
  res = a.split() 
  if i == 0:
    b0 = pd.DataFrame(res,columns=['topic1'])
  elif i == 1:
    b1 = pd.DataFrame(res,columns=['topic2']) 
  elif i == 2:
    b2 = pd.DataFrame(res,columns=['topic3'])
  else:
    b3 = pd.DataFrame(res,columns=['topic4'])
   
features = b0.join(b1)
features = features.join(b2)
features = features.join(b3)

In [ ]:
features =  (pd.concat([features, features.unstack().reset_index(drop=True).rename('all features')], axis=1))


In [ ]:
all_aspects = features['all features']
all_aspects = pd.DataFrame(all_aspects)



In [ ]:
x=[]
for i in range(0,40):
  x.append(i)
print(x)

dct = {}
for i in x:
  dct [i]= []

print(dct)

In [ ]:
for i in range(0,40):
  for j in range (0, len(df2['review'])):
    if features['all features'][i] in df2['review'][j]:
      dct[i].append(df2['review'][j])



In [ ]:
import nltk
nltk.download('punkt')
nltk.download('movie_reviews')
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer


In [ ]:
y=[]
for i in range(0,40):
  y.append(i)
print(y)

dcti = {}
for i in y:
  dcti [i]= []

print(dcti)

In [ ]:
aspect0 = pd.DataFrame(dct[0], columns=[('feature')])
aspect1 = pd.DataFrame(dct[1], columns=[('feature')])
aspect2 = pd.DataFrame(dct[2], columns=[('feature')])
aspect3 = pd.DataFrame(dct[3], columns=[('feature')])
aspect4 = pd.DataFrame(dct[4], columns=[('feature')])
aspect5 = pd.DataFrame(dct[5], columns=[('feature')])
aspect6 = pd.DataFrame(dct[6], columns=[('feature')])
aspect7 = pd.DataFrame(dct[7], columns=[('feature')])
aspect8 = pd.DataFrame(dct[8], columns=[('feature')])
aspect9 = pd.DataFrame(dct[9], columns=[('feature')])
aspect10 = pd.DataFrame(dct[10], columns=[('feature')])
aspect11 = pd.DataFrame(dct[11], columns=[('feature')])
aspect12 = pd.DataFrame(dct[12], columns=[('feature')])
aspect13 = pd.DataFrame(dct[13], columns=[('feature')])
aspect14 = pd.DataFrame(dct[14], columns=[('feature')])
aspect15 = pd.DataFrame(dct[15], columns=[('feature')])
aspect16 = pd.DataFrame(dct[16], columns=[('feature')])
aspect17= pd.DataFrame(dct[17], columns=[('feature')])
aspect18 = pd.DataFrame(dct[18], columns=[('feature')])
aspect19 = pd.DataFrame(dct[19], columns=[('feature')])
aspect20 = pd.DataFrame(dct[20], columns=[('feature')])
aspect21 = pd.DataFrame(dct[21], columns=[('feature')])
aspect22 = pd.DataFrame(dct[22], columns=[('feature')])
aspect23 = pd.DataFrame(dct[23], columns=[('feature')])
aspect24 = pd.DataFrame(dct[24], columns=[('feature')])
aspect25 = pd.DataFrame(dct[25], columns=[('feature')])
aspect26 = pd.DataFrame(dct[26], columns=[('feature')])
aspect27 = pd.DataFrame(dct[27], columns=[('feature')])
aspect28 = pd.DataFrame(dct[28], columns=[('feature')])
aspect29 = pd.DataFrame(dct[29], columns=[('feature')])
aspect30 = pd.DataFrame(dct[30], columns=[('feature')])
aspect31 = pd.DataFrame(dct[31], columns=[('feature')])
aspect32 = pd.DataFrame(dct[32], columns=[('feature')])
aspect33 = pd.DataFrame(dct[33], columns=[('feature')])
aspect34 = pd.DataFrame(dct[34], columns=[('feature')])
aspect35 = pd.DataFrame(dct[35], columns=[('feature')])
aspect36 = pd.DataFrame(dct[36], columns=[('feature')])
aspect37 = pd.DataFrame(dct[37], columns=[('feature')])
aspect38 = pd.DataFrame(dct[38], columns=[('feature')])
aspect39 = pd.DataFrame(dct[39], columns=[('feature')])



In [ ]:
for i in range (len(aspect0['feature'])):
  blob = TextBlob(aspect0['feature'][i], analyzer=None,)
  dcti[0].append(blob.sentiment)

for i in range (len(aspect1['feature'])):
  blob = TextBlob(aspect1['feature'][i], analyzer=None,)
  dcti[1].append(blob.sentiment)

for i in range (len(aspect2['feature'])):
  blob = TextBlob(aspect2['feature'][i], analyzer=None,)
  dcti[2].append(blob.sentiment)

for i in range (len(aspect3['feature'])):
  blob = TextBlob(aspect3['feature'][i], analyzer=None,)
  dcti[3].append(blob.sentiment)

for i in range (len(aspect4['feature'])):
  blob = TextBlob(aspect4['feature'][i], analyzer=None,)
  dcti[4].append(blob.sentiment)

for i in range (len(aspect5['feature'])):
  blob = TextBlob(aspect5['feature'][i], analyzer=None,)
  dcti[5].append(blob.sentiment)

for i in range (len(aspect6['feature'])):
  blob = TextBlob(aspect6['feature'][i], analyzer=None,)
  dcti[6].append(blob.sentiment)

for i in range (len(aspect7['feature'])):
  blob = TextBlob(aspect7['feature'][i], analyzer=None,)
  dcti[7].append(blob.sentiment)

for i in range (len(aspect8['feature'])):
  blob = TextBlob(aspect8['feature'][i], analyzer=None,)
  dcti[8].append(blob.sentiment)

for i in range (len(aspect9['feature'])):
  blob = TextBlob(aspect9['feature'][i], analyzer=None,)
  dcti[9].append(blob.sentiment)

for i in range (len(aspect10['feature'])):
  blob = TextBlob(aspect10['feature'][i], analyzer=None,)
  dcti[10].append(blob.sentiment)

for i in range (len(aspect11['feature'])):
  blob = TextBlob(aspect11['feature'][i], analyzer=None,)
  dcti[11].append(blob.sentiment)


for i in range (len(aspect12['feature'])):
  blob = TextBlob(aspect12['feature'][i], analyzer=None,)
  dcti[12].append(blob.sentiment)

for i in range (len(aspect13['feature'])):
  blob = TextBlob(aspect13['feature'][i], analyzer=None,)
  dcti[13].append(blob.sentiment)

for i in range (len(aspect14['feature'])):
  blob = TextBlob(aspect14['feature'][i], analyzer=None,)
  dcti[14].append(blob.sentiment)

for i in range (len(aspect15['feature'])):
  blob = TextBlob(aspect15['feature'][i], analyzer=None,)
  dcti[15].append(blob.sentiment)

for i in range (len(aspect16['feature'])):
  blob = TextBlob(aspect16['feature'][i], analyzer=None,)
  dcti[16].append(blob.sentiment)

for i in range (len(aspect17['feature'])):
  blob = TextBlob(aspect17['feature'][i], analyzer=None,)
  dcti[17].append(blob.sentiment)

for i in range (len(aspect18['feature'])):
  blob = TextBlob(aspect18['feature'][i], analyzer=None,)
  dcti[18].append(blob.sentiment)

for i in range (len(aspect19['feature'])):
  blob = TextBlob(aspect19['feature'][i], analyzer=None,)
  dcti[19].append(blob.sentiment)

for i in range (len(aspect20['feature'])):
  blob = TextBlob(aspect20['feature'][i], analyzer=None,)
  dcti[20].append(blob.sentiment)


for i in range (len(aspect21['feature'])):
  blob = TextBlob(aspect21['feature'][i], analyzer=None,)
  dcti[21].append(blob.sentiment)

for i in range (len(aspect22['feature'])):
  blob = TextBlob(aspect22['feature'][i], analyzer=None,)
  dcti[22].append(blob.sentiment)

for i in range (len(aspect23['feature'])):
  blob = TextBlob(aspect23['feature'][i], analyzer=None,)
  dcti[23].append(blob.sentiment)

for i in range (len(aspect24['feature'])):
  blob = TextBlob(aspect24['feature'][i], analyzer=None,)
  dcti[24].append(blob.sentiment)

for i in range (len(aspect25['feature'])):
  blob = TextBlob(aspect25['feature'][i], analyzer=None,)
  dcti[25].append(blob.sentiment)

for i in range (len(aspect26['feature'])):
  blob = TextBlob(aspect26['feature'][i], analyzer=None,)
  dcti[26].append(blob.sentiment)


for i in range (len(aspect27['feature'])):
  blob = TextBlob(aspect27['feature'][i], analyzer=None,)
  dcti[27].append(blob.sentiment)
  
for i in range (len(aspect28['feature'])):
  blob = TextBlob(aspect28['feature'][i], analyzer=None,)
  dcti[28].append(blob.sentiment)

for i in range (len(aspect29['feature'])):
  blob = TextBlob(aspect29['feature'][i], analyzer=None,)
  dcti[29].append(blob.sentiment)

for i in range (len(aspect30['feature'])):
  blob = TextBlob(aspect30['feature'][i], analyzer=None,)
  dcti[30].append(blob.sentiment)

for i in range (len(aspect31['feature'])):
  blob = TextBlob(aspect31['feature'][i], analyzer=None,)
  dcti[31].append(blob.sentiment)

for i in range (len(aspect32['feature'])):
  blob = TextBlob(aspect32['feature'][i], analyzer=None,)
  dcti[32].append(blob.sentiment)

for i in range (len(aspect33['feature'])):
  blob = TextBlob(aspect33['feature'][i], analyzer=None,)
  dcti[33].append(blob.sentiment)

for i in range (len(aspect34['feature'])):
  blob = TextBlob(aspect34['feature'][i], analyzer=None,)
  dcti[34].append(blob.sentiment)

for i in range (len(aspect35['feature'])):
  blob = TextBlob(aspect35['feature'][i], analyzer=None,)
  dcti[35].append(blob.sentiment)

for i in range (len(aspect36['feature'])):
  blob = TextBlob(aspect36['feature'][i], analyzer=None,)
  dcti[36].append(blob.sentiment)

for i in range (len(aspect37['feature'])):
  blob = TextBlob(aspect37['feature'][i], analyzer=None,)
  dcti[37].append(blob.sentiment)

for i in range (len(aspect38['feature'])):
  blob = TextBlob(aspect38['feature'][i], analyzer=None,)
  dcti[38].append(blob.sentiment)

for i in range (len(aspect39['feature'])):
  blob = TextBlob(aspect39['feature'][i], analyzer=None,)
  dcti[39].append(blob.sentiment)



In [ ]:
size = []
for i in range(0,40):
  size.append(len(dcti[i]))


In [ ]:
aspect0  = pd.DataFrame(dcti[0])
aspect0 = aspect0.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect0['mean'] = aspect0['polarity'].rolling(100).mean()


aspect1  = pd.DataFrame(dcti[1])
aspect1 = aspect1.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect1['mean'] = aspect1['polarity'].rolling(100).mean()

aspect2  = pd.DataFrame(dcti[2])
aspect2 = aspect2.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect2['mean'] = aspect2['polarity'].rolling(100).mean()

aspect3  = pd.DataFrame(dcti[3])
aspect3 = aspect3.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect3['mean'] = aspect3['polarity'].rolling(100).mean()


aspect4  = pd.DataFrame(dcti[4])
aspect4 = aspect4.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect4['mean'] = aspect4['polarity'].rolling(100).mean()

aspect5  = pd.DataFrame(dcti[5])
aspect5 = aspect5.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect5['mean'] = aspect5['polarity'].rolling(100).mean()

aspect6  = pd.DataFrame(dcti[6])
aspect6 = aspect6.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect6['mean'] = aspect6['polarity'].rolling(100).mean()

aspect7  = pd.DataFrame(dcti[7])
aspect7 = aspect7.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect7['mean'] = aspect7['polarity'].rolling(100).mean()

aspect8  = pd.DataFrame(dcti[8])
aspect8 = aspect8.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect8['mean'] = aspect8['polarity'].rolling(100).mean()

aspect9  = pd.DataFrame(dcti[9])
aspect9 = aspect9.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect9['mean'] = aspect9['polarity'].rolling(100).mean()

aspect10  = pd.DataFrame(dcti[10])
aspect10 = aspect10.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect10['mean'] = aspect10['polarity'].rolling(100).mean()

aspect11  = pd.DataFrame(dcti[11])
aspect11 = aspect11.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect11['mean'] = aspect11['polarity'].rolling(100).mean()

aspect12  = pd.DataFrame(dcti[12])
aspect12 = aspect12.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect12['mean'] = aspect12['polarity'].rolling(100).mean()

aspect13  = pd.DataFrame(dcti[13])
aspect13 = aspect13.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect13['mean'] = aspect13['polarity'].rolling(100).mean()

aspect14  = pd.DataFrame(dcti[14])
aspect14 = aspect14.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect14['mean'] = aspect14['polarity'].rolling(100).mean()

aspect15  = pd.DataFrame(dcti[15])
aspect15 = aspect15.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect15['mean'] = aspect15['polarity'].rolling(100).mean()

aspect16  = pd.DataFrame(dcti[16])
aspect16 = aspect16.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect16['mean'] = aspect16['polarity'].rolling(100).mean()

aspect17  = pd.DataFrame(dcti[17])
aspect17 = aspect17.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect17['mean'] = aspect17['polarity'].rolling(100).mean()

aspect18  = pd.DataFrame(dcti[18])
aspect18 = aspect18.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect18['mean'] = aspect18['polarity'].rolling(100).mean()


aspect19  = pd.DataFrame(dcti[19])
aspect19 = aspect19.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect19['mean'] = aspect19['polarity'].rolling(100).mean()

aspect20 = pd.DataFrame(dcti[20])
aspect20 = aspect20.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect20['mean'] = aspect20['polarity'].rolling(100).mean()


aspect21  = pd.DataFrame(dcti[21])
aspect21 = aspect21.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect21['mean'] = aspect21['polarity'].rolling(100).mean()

aspect22  = pd.DataFrame(dcti[22])
aspect22 = aspect22.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect22['mean'] = aspect22['polarity'].rolling(100).mean()

aspect23  = pd.DataFrame(dcti[23])
aspect23 = aspect23.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect23['mean'] = aspect23['polarity'].rolling(100).mean()


aspect24  = pd.DataFrame(dcti[24])
aspect24 = aspect24.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect24['mean'] = aspect24['polarity'].rolling(100).mean()


aspect25  = pd.DataFrame(dcti[25])
aspect25 = aspect25.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect25['mean'] = aspect25['polarity'].rolling(100).mean()

aspect26  = pd.DataFrame(dcti[26])
aspect26 = aspect26.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect26['mean'] = aspect26['polarity'].rolling(100).mean()

aspect27  = pd.DataFrame(dcti[27])
aspect27 = aspect27.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect27['mean'] = aspect27['polarity'].rolling(100).mean()

aspect28  = pd.DataFrame(dcti[28])
aspect28 = aspect28.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect28['mean'] = aspect28['polarity'].rolling(100).mean()


aspect29  = pd.DataFrame(dcti[29])
aspect29 = aspect29.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect29['mean'] = aspect29['polarity'].rolling(100).mean()

aspect30  = pd.DataFrame(dcti[30])
aspect30 = aspect30.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect30['mean'] = aspect30['polarity'].rolling(100).mean()


aspect31  = pd.DataFrame(dcti[31])
aspect31 = aspect31.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect31['mean'] = aspect31['polarity'].rolling(100).mean()

aspect32  = pd.DataFrame(dcti[32])
aspect32 = aspect32.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect32['mean'] = aspect32['polarity'].rolling(100).mean()

aspect33  = pd.DataFrame(dcti[33])
aspect33 = aspect33.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect33['mean'] = aspect33['polarity'].rolling(100).mean()

aspect34  = pd.DataFrame(dcti[34])
aspect34 = aspect34.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect34['mean'] = aspect34['polarity'].rolling(100).mean()


aspect35 = pd.DataFrame(dcti[35])
aspect35 = aspect35.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect35['mean'] = aspect35['polarity'].rolling(100).mean()

aspect36 = pd.DataFrame(dcti[36])
aspect36 = aspect36.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect36['mean'] = aspect36['polarity'].rolling(100).mean()

aspect37  = pd.DataFrame(dcti[37])
aspect37 = aspect37.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect37['mean'] = aspect37['polarity'].rolling(100).mean()

aspect38  = pd.DataFrame(dcti[38])
aspect38 = aspect38.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect38['mean'] = aspect38['polarity'].rolling(100).mean()

aspect39  = pd.DataFrame(dcti[39])
aspect39 = aspect39.sort_values(by='subjectivity', ascending=False, na_position='first')
aspect39['mean'] = aspect39['polarity'].rolling(100).mean()

In [ ]:
sentiment = []


aspect0 = aspect0.dropna()
aspect0 = aspect0.reset_index(drop=True)
aspect0['mean'][0]

if aspect0['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')


aspect1 = aspect1.dropna()
aspect1 = aspect1.reset_index(drop=True)
aspect1['mean'][0]

if aspect1['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')


aspect2 = aspect2.dropna()
aspect2 = aspect2.reset_index(drop=True)
aspect2['mean'][0]

if aspect2['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')
  

aspect3 = aspect3.dropna()
aspect3 = aspect3.reset_index(drop=True)
aspect3['mean'][0]

if aspect3['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect4 = aspect4.dropna()
aspect4 = aspect4.reset_index(drop=True)
aspect4['mean'][0]

if aspect4['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')


aspect5 = aspect5.dropna()
aspect5 = aspect5.reset_index(drop=True)
aspect5['mean'][0]

if aspect5['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect6 = aspect6.dropna()
aspect6 = aspect6.reset_index(drop=True)
aspect6['mean'][0]

if aspect6['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')


aspect7 = aspect7.dropna()
aspect7 = aspect7.reset_index(drop=True)
aspect7['mean'][0]


if aspect7['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')


aspect8 = aspect8.dropna()
aspect8 = aspect8.reset_index(drop=True)
aspect8['mean'][0]

if aspect8['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect9 = aspect9.dropna()
aspect9 = aspect9.reset_index(drop=True)
aspect9['mean'][0]

if aspect9['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')


aspect10 = aspect10.dropna()
aspect10 = aspect10.reset_index(drop=True)
aspect10['mean'][0]


if aspect10['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')



aspect11 = aspect11.dropna()
aspect11 = aspect11.reset_index(drop=True)
aspect11['mean'][0]

if aspect11['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')



aspect12 = aspect12.dropna()
aspect12 = aspect12.reset_index(drop=True)
aspect12['mean'][0]

if aspect12['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect13 = aspect13.dropna()
aspect13 = aspect13.reset_index(drop=True)
aspect13['mean'][0]


if aspect13['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect14 = aspect14.dropna()
aspect14 = aspect14.reset_index(drop=True)
aspect14['mean'][0]

if aspect14['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect15 = aspect15.dropna()
aspect15 = aspect15.reset_index(drop=True)
aspect15['mean'][0]

if aspect15['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')


aspect16 = aspect16.dropna()
aspect16 = aspect16.reset_index(drop=True)
aspect16['mean'][0]

if aspect16['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')


aspect17 = aspect17.dropna()
aspect17 = aspect17.reset_index(drop=True)
aspect17['mean'][0]

if aspect17['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect18 = aspect18.dropna()
aspect18 = aspect18.reset_index(drop=True)
aspect18['mean'][0]

if aspect18['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect19 = aspect19.dropna()
aspect19 = aspect19.reset_index(drop=True)
aspect19['mean'][0]

if aspect19['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect20 = aspect20.dropna()
aspect20 = aspect20.reset_index(drop=True)
aspect20['mean'][0]


if aspect20['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')
  


aspect21 = aspect21.dropna()
aspect21 = aspect21.reset_index(drop=True)
aspect21['mean'][0]

if aspect21['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect22 = aspect22.dropna()
aspect22 = aspect22.reset_index(drop=True)
aspect22['mean'][0]


if aspect22['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect23 = aspect23.dropna()
aspect23 = aspect23.reset_index(drop=True)
aspect23['mean'][0]


if aspect23['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect24 = aspect24.dropna()
aspect24 = aspect24.reset_index(drop=True)
aspect24['mean'][0]

if aspect24['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')


aspect25 = aspect25.dropna()
aspect25 = aspect25.reset_index(drop=True)
aspect25['mean'][0]


if aspect25['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect26 = aspect26.dropna()
aspect26 = aspect26.reset_index(drop=True)
aspect26['mean'][0]

if aspect26['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect27 = aspect27.dropna()
aspect27 = aspect27.reset_index(drop=True)
aspect27['mean'][0]

if aspect27['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect28 = aspect28.dropna()
aspect28 = aspect28.reset_index(drop=True)
aspect28['mean'][0]

if aspect28['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect29 = aspect29.dropna()
aspect29 = aspect29.reset_index(drop=True)
aspect29['mean'][0]

if aspect29['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect30 = aspect30.dropna()
aspect30 = aspect30.reset_index(drop=True)
aspect30['mean'][0]

if aspect30['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')
  

aspect31 = aspect31.dropna()
aspect31 = aspect31.reset_index(drop=True)
aspect31['mean'][0]

if aspect31['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect32 = aspect32.dropna()
aspect32 = aspect32.reset_index(drop=True)
aspect32['mean'][0]


if aspect32['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect33 = aspect33.dropna()
aspect33 = aspect33.reset_index(drop=True)
aspect33['mean'][0]


if aspect33['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect34 = aspect34.dropna()
aspect34 = aspect34.reset_index(drop=True)
aspect34['mean'][0]


if aspect34['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect35 = aspect35.dropna()
aspect35 = aspect35.reset_index(drop=True)
aspect35['mean'][0]


if aspect35['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect36 = aspect36.dropna()
aspect36 = aspect36.reset_index(drop=True)
aspect36['mean'][0]

if aspect36['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect37 = aspect37.dropna()
aspect37 = aspect37.reset_index(drop=True)
aspect37['mean'][0]

if aspect37['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')


aspect38 = aspect38.dropna()
aspect38 = aspect38.reset_index(drop=True)
aspect38['mean'][0]


if aspect38['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')

aspect39 = aspect39.dropna()
aspect39 = aspect39.reset_index(drop=True)
aspect39['mean'][0]


if aspect39['mean'][0] > 0.4:
  sentiment.append('positive')
else:
  sentiment.append('negative')





In [ ]:
all_aspects['sentiment'] = pd.DataFrame(sentiment)
all_aspects['number of review'] = pd.DataFrame(size)


In [ ]:
all_aspects

In [ ]:
all_aspects['sentiment'].value_counts()

In [ ]:
# # Visualize the topics
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
# vis
